In [56]:
import pandas as pd 
from pathlib import Path

In [57]:
dataset_path='../../DRIM/data/TCGA/GBMLGG/raw_RNA/'

In [58]:
sample_id ='00362664-cb13-4998-a26c-618dba69ea00'

In [59]:
sample_data_path =Path(dataset_path) / sample_id / 'a7732b3f-6294-47ec-b869-0aac6ee6c815.rna_seq.augmented_star_gene_counts.tsv'

In [60]:
sample_data = pd.read_table(sample_data_path)

In [61]:
sample_data

,,,,,,,,# gene-model: GENCODE v36
gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
N_unmapped,NaN,NaN,2800310,2800310,2800310,NaN,NaN,NaN
N_multimapping,NaN,NaN,6575856,6575856,6575856,NaN,NaN,NaN
N_noFeature,NaN,NaN,4096328,42598823,42961358,NaN,NaN,NaN
N_ambiguous,NaN,NaN,8182106,1964691,1946794,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
ENSG00000288669.1,AC008763.4,protein_coding,0,1,0,0.0000,0.0000,0.0000
ENSG00000288670.1,AL592295.6,lncRNA,1201,632,655,30.1440,9.3454,9.6219
ENSG00000288671.1,AC006486.3,protein_coding,0,0,0,0.0000,0.0000,0.0000
ENSG00000288674.1,AL391628.1,protein_coding,5,2,3,0.0236,0.0073,0.0075


## Воспользуемся препроцессингом из прошлой статьи

In [62]:
# import needed libraries
import pandas as pd
import requests
import io
import pandas as pd
import os
import numpy as np
from collections import OrderedDict
import itertools
from typing import Tuple, Dict
import tqdm
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from IPython import display
# import pyvips
import torch
import torch.nn.functional as F
import cv2
import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import torchvision
from torchvision.transforms import ToTensor
import torch
import json

base_path = '../../DRIM/data/TCGA/GBMLGG/'

In [63]:
!nvidia-smi

Thu Dec  5 12:07:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off | 00000000:38:00.0 Off |                  Off |
| 30%   27C    P8              24W / 300W |      4MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [64]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install opencv-python

In [65]:
dataframe = pd.read_csv('../../DRIM/data/files/clinical_data.tsv', sep='\t')
dataframe.shape

(1104, 18)

In [11]:
dataframe = dataframe[dataframe.project_id.isin(['GBM', 'LGG'])]
dataframe.shape

(1104, 18)

In [12]:
dataframe['group'].value_counts()

group
train    886
test     218
Name: count, dtype: int64

In [13]:
# helpers function
def request_file_info(data_type):
    """
    Adapted from Vale-Silva's preprocessing.
    """
    fields = [
        'file_name',
        'cases.submitter_id',
        'cases.samples.sample_type',
        'cases.project.project_id',
        'cases.project.primary_site',
        ]

    fields = ','.join(fields)

    files_endpt = 'https://api.gdc.cancer.gov/files'

    filters = {
        'op': 'and',
        'content':[
             {
        "op": "in",
        "content":{
            "field": "cases.project.primary_site",
            "value": ["Brain"]
            }
        },{
            'op': 'in',
            'content':{
                'field': 'files.experimental_strategy',
                'value': [data_type]
                }
            }  
        ]
    }

    params = {
        'filters': filters,
        'fields': fields,
        'format': 'TSV',
        'size': '200000'
        }

    response = requests.post(files_endpt, headers = {'Content-Type': 'application/json'}, json = params)

    return pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')

In [14]:
def make_patient_file_map(df, base_dir):
    """
    Adapted from Vale-Silva's preprocessing.
    """
    
    d = {}
    for _, row in df.iterrows():
        patient = row['cases.0.submitter_id']
        if os.path.exists(os.path.join(base_dir, row.id, row.file_name)):
            if patient in d:
                if not isinstance(d[patient], tuple):
                    d[patient] = (
                        d[patient],
                        os.path.join(base_dir, row.id, row.file_name))
                else:
                    d[patient] += os.path.join(base_dir, row.id, row.file_name),
            else:
                d[patient] = os.path.join(base_dir, row.id, row.file_name)

    return d

# 2. Clinical Data <a name="p2"></a>

In [15]:
%%capture
supplementary = pd.read_csv('../../DRIM/data/files/supplementary_data.tsv', sep='\t')

In [16]:
mapper_clin = {
    'patient': 'submitter_id', 
    'IDH.status': 'IDH', 
    'MGMT.promoter.status': 'MGMT',
    'X1p.19q.codeletion': 'X1p19q',
    'Chr.19.20.co.gain': '19.20.gain',
    'Chr.7.gain.Chr.10.loss': '7g10l',
    'TERT.expression.status': 'TERT',
    'ATRX.status': 'ATRX',
}

In [17]:
supplementary = supplementary.drop(columns=['IDH', 'Mutations', 'TERT']).rename(columns=mapper_clin)[list(mapper_clin.values())]
supplementary = supplementary[~supplementary.duplicated(subset=['submitter_id'], keep='first')].dropna(subset='submitter_id')

In [18]:
dataframe = dataframe.merge(supplementary, how='left', on='submitter_id')
dataframe.shape

(1104, 25)

In [19]:
for col in list(mapper_clin.values()):
    print(dataframe[col].value_counts())

submitter_id
TCGA-CS-4938    1
TCGA-06-5415    1
TCGA-06-5859    1
TCGA-06-5858    1
TCGA-06-5856    1
               ..
TCGA-P5-A77X    1
TCGA-P5-A780    1
TCGA-P5-A781    1
TCGA-QH-A65R    1
TCGA-RR-A6KC    1
Name: count, Length: 1104, dtype: int64
IDH
WT        529
Mutant    453
Name: count, dtype: int64
MGMT
Methylated      607
Unmethylated    311
Name: count, dtype: int64
X1p19q
non-codel    913
codel        169
Name: count, dtype: int64
19.20.gain
No chr 19/20 gain    969
Gain chr 19/20       104
Name: count, dtype: int64
7g10l
No combined CNA             644
Gain chr 7 & loss chr 10    429
Name: count, dtype: int64
TERT
Expressed        348
Not expressed    312
Name: count, dtype: int64
ATRX
WT        601
Mutant    208
Name: count, dtype: int64


In [20]:
dataframe

,Unnamed: 0,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,...,time,event,group,IDH,MGMT,X1p19q,19.20.gain,7g10l,TERT,ATRX
0,607,TCGA-CS-4938,NaN,11509.0,No,no,No,3574.0,female,white,...,9.791781,0,train,Mutant,Unmethylated,non-codel,No chr 19/20 gain,No combined CNA,Not expressed,Mutant
1,608,TCGA-CS-4941,NaN,24578.0,No,no,No,NaN,male,white,...,0.641096,1,train,WT,Methylated,non-codel,No chr 19/20 gain,Gain chr 7 & loss chr 10,Expressed,WT
2,609,TCGA-CS-4942,NaN,16297.0,No,no,No,NaN,female,black or african american,...,3.657534,1,train,Mutant,Unmethylated,non-codel,No chr 19/20 gain,No combined CNA,Not expressed,Mutant
3,610,TCGA-CS-4943,NaN,13565.0,No,no,No,NaN,male,white,...,3.030137,1,train,Mutant,Methylated,non-codel,No chr 19/20 gain,No combined CNA,Not expressed,Mutant
4,611,TCGA-CS-4944,NaN,18494.0,No,no,No,1828.0,male,white,...,5.008219,0,train,Mutant,Methylated,non-codel,No chr 19/20 gain,No combined CNA,Expressed,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,4475,TCGA-87-5896,NaN,18391.0,No,NaN,NaN,800.0,female,white,...,2.191781,0,test,WT,Unmethylated,non-codel,No chr 19/20 gain,Gain chr 7 & loss chr 10,NaN,WT
1100,4476,TCGA-OX-A56R,NaN,25178.0,No,no,No,NaN,male,black or african american,...,0.493151,1,train,WT,Methylated,non-codel,No chr 19/20 gain,Gain chr 7 & loss chr 10,NaN,WT
1101,4477,TCGA-RR-A6KA,NaN,26582.0,No,no,No,NaN,female,black or african american,...,0.523288,1,train,WT,Methylated,non-codel,Gain chr 19/20,No combined CNA,NaN,WT
1102,4478,TCGA-RR-A6KB,NaN,18713.0,No,no,No,0.0,male,black or african american,...,0.000000,0,train,WT,Unmethylated,non-codel,No chr 19/20 gain,Gain chr 7 & loss chr 10,NaN,WT


# 4. RNASeq <a name="p4"></a>

In [29]:
%%time

mRNA_files = request_file_info(data_type='RNA-Seq')
mRNA_files.shape

CPU times: user 78.5 ms, sys: 1.01 ms, total: 79.5 ms
Wall time: 3.38 s


(6390, 6)

In [30]:
mRNA_files = mRNA_files[mRNA_files['cases.0.project.project_id'].str.startswith('TCGA')]
mRNA_files = mRNA_files[mRNA_files['file_name'].str.endswith('rna_seq.augmented_star_gene_counts.tsv')]
mRNA_files = mRNA_files[mRNA_files['cases.0.samples.0.sample_type'] == 'Primary Tumor']
mRNA_files.shape

(674, 6)

In [31]:
print('All rows:       ', mRNA_files.shape[0])
print('Unique patients:', mRNA_files['cases.0.submitter_id'].unique().shape[0])

All rows:        674
Unique patients: 672


In [32]:
mRNA_files = mRNA_files[~mRNA_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]
mRNA_files.shape

(672, 6)

In [33]:
file_map = make_patient_file_map(mRNA_files, os.path.join(base_path, 'raw_RNA'))
file_map = {k: file_map[k] for k in file_map if k in list(dataframe['submitter_id'])}
len(file_map)

664

In [34]:
RNA_mapping = pd.DataFrame([(k, v) for k, v in file_map.items()], columns=('submitter_id', 'RNA'))
dataframe = dataframe.merge(RNA_mapping, how='left', on='submitter_id')

In [37]:
dataframe.RNA.notnull().sum()

664

## Preprocessing
In this section, we only consider genes coding for proteins, and we also remove genes with a variance of less than 0.1 on the training set.

In [38]:
def load_single_file_RNA(path):
    temp_df = pd.read_csv(path, sep='\t', skiprows=6, usecols=[0, 2, 8], names=['gene_id', 'gene_type', 'fpkm_uq_unstranded']).set_index('gene_id')#, 2, 3, 4, 5, 6, 7, 8])
    temp_df = temp_df[temp_df['gene_type'] == 'protein_coding']['fpkm_uq_unstranded']
    return temp_df

dataframe_train = dataframe[dataframe.group == 'train']
file_map_train = make_patient_file_map(mRNA_files, os.path.join(base_path, 'raw_RNA'))
file_map_train = {k: file_map[k] for k in file_map if k in list(dataframe_train['submitter_id'])}
len(file_map_train)

532

In [39]:
rna_train_dataframe = pd.concat([load_single_file_RNA(path) for path in file_map_train.values()], axis=1)

In [40]:
gene_description = rna_train_dataframe.T.describe()

In [41]:
gene_description

gene_id,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288649.1,ENSG00000288654.1,ENSG00000288656.1,ENSG00000288658.1,ENSG00000288660.1,ENSG00000288661.1,ENSG00000288669.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
count,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,...,532.0,532.0,532.000000,532.000000,532.000000,532.0,532.000000,532.0,532.000000,532.000000
mean,16.516526,0.460505,19.835028,1.684326,0.949447,3.260354,3.056973,8.886332,8.401908,7.555184,...,0.0,0.0,0.001792,0.439711,0.022902,0.0,0.000729,0.0,0.010868,0.710279
std,9.744088,3.821636,6.563378,0.341117,0.524027,2.503685,9.265542,6.750917,4.541117,2.408429,...,0.0,0.0,0.003519,0.603313,0.036959,0.0,0.002916,0.0,0.008565,0.471296
min,1.670300,0.000000,8.945500,0.767800,0.186600,0.543800,0.195000,0.333800,0.835500,1.897700,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.082200
25%,11.430450,0.037075,15.757025,1.455200,0.585150,1.742625,0.796500,4.073275,5.046775,5.982375,...,0.0,0.0,0.000000,0.113150,0.000000,0.0,0.000000,0.0,0.005675,0.426200
50%,14.637200,0.073200,18.042500,1.655000,0.833700,2.568250,1.432950,6.424700,7.589100,7.140350,...,0.0,0.0,0.000000,0.224850,0.000000,0.0,0.000000,0.0,0.009400,0.612950
75%,19.533200,0.144800,21.810175,1.850950,1.165775,3.915625,2.930925,12.461325,10.708500,8.606525,...,0.0,0.0,0.003625,0.508450,0.032225,0.0,0.000000,0.0,0.014425,0.886450
max,112.397800,67.389800,62.112300,3.623200,4.044100,25.299400,170.688200,36.041900,46.288400,20.948600,...,0.0,0.0,0.020900,4.768300,0.298400,0.0,0.042600,0.0,0.118600,5.283000


In [42]:
genes_to_keep = gene_description.columns[gene_description.iloc[2, :] > 0.1].values.tolist()

In [43]:
def load_preprocessed_single_file_RNA(path, genes_to_keep=genes_to_keep):
    temp_df = load_single_file_RNA(path)
    return temp_df.loc[genes_to_keep]

In [45]:
len(genes_to_keep)

16304

In [46]:
output_path = os.path.join(base_path, 'RNA')

if not os.path.exists(output_path):
    os.makedirs(output_path)

for patient, filepath in tqdm.tqdm(file_map.items()):
    idx, filename = filepath.split('/')[-2:]
    if not os.path.exists(os.path.join(output_path, idx)):
        os.makedirs(os.path.join(output_path, idx))
        
    temp_df = load_preprocessed_single_file_RNA(filepath)
    temp_df.to_csv(os.path.join(output_path, idx, filename.replace('tsv', 'csv')))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 664/664 [00:57<00:00, 11.45it/s]


In [48]:
def replace_rna(x):
    if pd.isna(x):
        return x
    else:
        return str(x).replace('raw_RNA', 'RNA').replace('tsv', 'csv')
    
    
dataframe['RNA'] = dataframe['RNA'].apply(replace_rna)

In [51]:
dataframe.to_csv('../../DRIM/data/files/dataset.csv', index=False)